# From Detection to Credibility: A Machine Learning Framework for Assessing News Source Reliability



In [1]:
# !pip3 install -r requirements.txt
!pip3 install tensorflow

  Obtaining dependency information for numpy<2.0.0,>=1.23.5 from https://files.pythonhosted.org/packages/3f/6b/5610004206cf7f8e7ad91c5a85a8c71b2f2f8051a0c0c4d5916b76d6cbb2/numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blis 1.0.1 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\Ryan\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [1]:
# Import necessary libraries

# Data manipulation
import pandas as pd
import numpy as np

# Statistical functions
from scipy.stats import zscore

# For concurrency (running functions in parallel)
from concurrent.futures import ThreadPoolExecutor

# For caching (to speed up repeated function calls)
from functools import lru_cache

# For progress tracking
from tqdm import tqdm

# Plotting and Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Language Detection packages
# `langdetect` for detecting language
from langdetect import detect as langdetect_detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
# `langid` for an alternative language detection method
from langid import classify as langid_classify

# Text Preprocessing and NLP
# Stopwords (common words to ignore) from NLTK
from nltk.corpus import stopwords
# Tokenizing sentences/words
from nltk.tokenize import word_tokenize
# Part-of-speech tagging
from nltk import pos_tag
# Lemmatization (converting words to their base form)
from nltk.stem import WordNetLemmatizer
import nltk
# Regular expressions for text pattern matching
import re

# Word Cloud generation
from wordcloud import WordCloud

In [2]:
data = pd.read_csv("../processed_data.csv")

Since our end goal is to determine the credibility of popular American news outlet websites, which uses the English language, we will drop rows where the language is NOT in English (`en`).


# Embedding layer + LSTM + gridsearch_cv + K-Fold Cross Validation


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l2


# Tokenize and pad sequences
vocab_size = 20000
max_len = 100
embedding_dim = 100

texts = data['processed_full_content'].values
labels = data['label'].values

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=max_len)

# Define LSTM model function for GridSearchCV with Early Stopping
def create_model(lstm_units=64, dropout_rate=0.2, l2_reg=0.01):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
        LSTM(lstm_units, return_sequences=True, kernel_regularizer=l2(l2_reg)),  # First LSTM with L2 regularization
        Dropout(dropout_rate),
        LSTM(lstm_units, return_sequences=False, kernel_regularizer=l2(l2_reg)),  # Last LSTM with L2 regularization
        Dense(32, activation='relu', kernel_regularizer=l2(l2_reg)),  # Dense layer with L2 regularization
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid', kernel_regularizer=l2(l2_reg))  # Output layer with L2 regularization
    ])
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Keras wrapper for scikit-learn compatibility, with more epochs
model = KerasClassifier(
    build_fn=create_model,
    epochs=50,
    verbose=1,
    callbacks=[early_stopping],
    validation_split=0.2
)

# Scoring metrics
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score)
}

# Grid search parameters
param_grid = {
    'lstm_units': [64, 128],     # 2 options
    'dropout_rate': [0.2, 0.3],  # 2 options

}

# Stratified K-Fold Cross Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, refit='accuracy', cv=skf)
grid_result = grid.fit(padded_sequences, labels)

2024-11-01 09:57:33.270152: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-01 09:57:33.324541: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-01 09:57:33.325361: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-01 09:57:34.892000: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/tmp/ipykernel_3204075/978346373.py:47: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(
2024-11-01 09:58:09.335574: W te

Epoch 1/50


2024-11-01 09:58:09.703205: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-11-01 09:58:09.704394: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-11-01 09:58:09.705248: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1278/1278 [==============================] - ETA: 0s - loss: 0.3994 - accuracy: 0.9086

2024-11-01 09:59:37.735776: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-11-01 09:59:37.736997: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-11-01 09:59:37.737934: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

...

2024-11-01 14:47:50.136386: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-11-01 14:47:50.138246: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-11-01 14:47:50.139714: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1597/1597 [==============================] - ETA: 0s - loss: 0.3630 - accuracy: 0.9164

2024-11-01 14:49:35.024503: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-11-01 14:49:35.026026: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-11-01 14:49:35.027268: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1597/1597 [==============================] - 113s 69ms/step - loss: 0.3630 - accuracy: 0.9164 - val_loss: 0.1928 - val_accuracy: 0.9476
Epoch 2/50
1597/1597 [==============================] - 109s 68ms/step - loss: 0.1678 - accuracy: 0.9583 - val_loss: 0.1709 - val_accuracy: 0.9523
Epoch 3/50
1597/1597 [==============================] - 109s 68ms/step - loss: 0.1407 - accuracy: 0.9681 - val_loss: 0.1823 - val_accuracy: 0.9482
Epoch 4/50
1597/1597 [==============================] - 110s 69ms/step - loss: 0.1169 - accuracy: 0.9760 - val_loss: 0.1712 - val_accuracy: 0.9529
Epoch 5/50
1597/1597 [==============================] - 110s 69ms/step - loss: 0.1038 - accuracy: 0.9798 - val_loss: 0.1865 - val_accuracy: 0.9551
Best Accuracy: 0.9466802380206701
Best Parameters: {'dropout_rate': 0.2, 'lstm_units': 64}


AttributeError: 'KerasClassifier' object has no attribute 'model_'

In [10]:
print(grid.best_estimator_.model.summary())


# Best results
print(f"\nBest Accuracy: {grid_result.best_score_}")
print(f"Best Parameters: {grid_result.best_params_}\n")

# Cross-validated metrics for best model

print("Detailed Metrics for Best Model:")
for metric in scoring.keys():
    print(f"{metric.capitalize()}: {grid_result.cv_results_['mean_test_' + metric][grid_result.best_index_]}")

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (None, 100, 100)          2000000   
                                                                 
 lstm_40 (LSTM)              (None, 100, 64)           42240     
                                                                 
 dropout_40 (Dropout)        (None, 100, 64)           0         
                                                                 
 lstm_41 (LSTM)              (None, 64)                33024     
                                                                 
 dense_40 (Dense)            (None, 32)                2080      
                                                                 
 dropout_41 (Dropout)        (None, 32)                0         
                                                                 
 dense_41 (Dense)            (None, 1)               

# LSTM + Word2Vec

In [3]:
import numpy as np
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from gensim.models import Word2Vec

# Train and process Word2Vec embeddings
def train_word2vec_embeddings(sentences, word_index, embedding_dim=100):
    word2vec = Word2Vec(sentences=sentences, vector_size=embedding_dim, window=5, min_count=1)
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        if word in word2vec.wv:
            embedding_matrix[i] = word2vec.wv[word]
    return embedding_matrix

# Define LSTM model with Dropout and L2 regularization
def create_lstm_model(vocab_size, embedding_matrix, input_length, learning_rate=0.001, l2_lambda=0.01):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_matrix.shape[1],
                        weights=[embedding_matrix],
                        input_length=input_length,
                        trainable=True))
    model.add(LSTM(units=64, return_sequences=False, dropout=0.2))
    model.add(Dropout(0.2))  # Added Dropout layer here
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(l2_lambda)))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate), metrics=['accuracy'])

    return model

# Early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Stratified K-Fold Cross-Validation with additional metrics
def stratified_k_fold_cross_validation(X, y, embedding_matrix, vocab_size, max_len, n_splits=5, batch_size=128):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'roc_auc': []}

    for train_idx, val_idx in skf.split(X, y):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        model = create_lstm_model(vocab_size, embedding_matrix, max_len, learning_rate=0.001, l2_lambda=0.01)
        
        # Fit model with early stopping
        model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=10,
            batch_size=batch_size,
            verbose=1,
            callbacks=[early_stopping]
        )
        
        y_pred = (model.predict(X_val) > 0.5).astype("int32")
        y_pred_prob = model.predict(X_val).ravel()

        accuracy = accuracy_score(y_val, y_pred)
        precision = precision_score(y_val, y_pred)
        recall = recall_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred)
        roc_auc = roc_auc_score(y_val, y_pred_prob)

        metrics['accuracy'].append(accuracy)
        metrics['precision'].append(precision)
        metrics['recall'].append(recall)
        metrics['f1'].append(f1)
        metrics['roc_auc'].append(roc_auc)

        print(f"Fold Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}, ROC AUC: {roc_auc}")

    avg_metrics = {metric: np.mean(scores) for metric, scores in metrics.items()}
    print("\nAverage Metrics Across Folds:")
    for metric, avg_score in avg_metrics.items():
        print(f"{metric.capitalize()}: {avg_score:.4f}")
    
    return avg_metrics

# Example usage
# Load your data and tokenize it
texts = data["processed_full_content"]
labels = data["label"].values

# Tokenize and prepare sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
X = tokenizer.texts_to_sequences(texts)
max_len = 100
X = pad_sequences(X, maxlen=max_len)

# Train Word2Vec on tokenized sequences
sentences = [text.split() for text in texts]  # Convert texts into tokenized sentences
embedding_dim = 100
embedding_matrix = train_word2vec_embeddings(sentences, tokenizer.word_index, embedding_dim)

# Perform Stratified K-Fold cross-validation
vocab_size = len(tokenizer.word_index) + 1
avg_metrics = stratified_k_fold_cross_validation(X, labels, embedding_matrix, vocab_size, max_len)
print("Final Average Metrics:", avg_metrics)


Epoch 1/10
400/400 [==============================] - 12s 24ms/step - loss: 0.2245 - accuracy: 0.9183 - val_loss: 0.1571 - val_accuracy: 0.9486
Epoch 2/10
400/400 [==============================] - 9s 23ms/step - loss: 0.1301 - accuracy: 0.9585 - val_loss: 0.1334 - val_accuracy: 0.9565
Epoch 3/10
400/400 [==============================] - 9s 23ms/step - loss: 0.1014 - accuracy: 0.9692 - val_loss: 0.1098 - val_accuracy: 0.9621
Epoch 4/10
400/400 [==============================] - 9s 23ms/step - loss: 0.0829 - accuracy: 0.9767 - val_loss: 0.1110 - val_accuracy: 0.9646
Epoch 5/10
400/400 [==============================] - 9s 23ms/step - loss: 0.0664 - accuracy: 0.9825 - val_loss: 0.1227 - val_accuracy: 0.9645
Epoch 6/10
400/400 [==============================] - 9s 23ms/step - loss: 0.0535 - accuracy: 0.9873 - val_loss: 0.1074 - val_accuracy: 0.9678
Epoch 7/10
400/400 [==============================] - 9s 23ms/step - loss: 0.0453 - accuracy: 0.9899 - val_loss: 0.1055 - val_accuracy: 0.968

# LSTM + GloVe 100D

In [4]:
import numpy as np
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


# Load and process GloVe embeddings
def load_glove_embeddings(glove_file, word_index, embedding_dim=100):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefficients = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefficients
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix


# Define LSTM model with Dropout and L2 regularization
def create_lstm_model(vocab_size, embedding_matrix, input_length, learning_rate=0.001, l2_lambda=0.01):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_matrix.shape[1],
                        weights=[embedding_matrix],
                        input_length=input_length,
                        trainable=True))
    model.add(LSTM(units=64, return_sequences=False, dropout=0.2))
    model.add(Dropout(0.2))  # Added Dropout layer here
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(l2_lambda)))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate), metrics=['accuracy'])

    return model
early_stopping = EarlyStopping(
    monitor='val_loss',  # or 'val_accuracy' if you prefer to monitor accuracy
    patience=3,          # stop after 3 epochs with no improvement
    restore_best_weights=True  # revert to the best weights after stopping
)
# K-Fold Cross-Validation with additional metrics
def k_fold_cross_validation(X, y, embedding_matrix, vocab_size, max_len, n_splits=5, batch_size=128):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'roc_auc': []}

    # Define early stopping
    early_stopping = EarlyStopping(
        monitor='val_loss',  # monitor validation loss
        patience=3,          # number of epochs with no improvement
        restore_best_weights=True  # revert to the best model weights
    )

    for train_idx, val_idx in kfold.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        model = create_lstm_model(vocab_size, embedding_matrix, max_len, learning_rate=0.001, l2_lambda=0.01)
        
        # Fit model with early stopping
        model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=10,             # maximum number of epochs
            batch_size=128,
            verbose=1,
            callbacks=[early_stopping]  # add early stopping
        )
        
        y_pred = (model.predict(X_val) > 0.5).astype("int32")
        y_pred_prob = model.predict(X_val).ravel()

        accuracy = accuracy_score(y_val, y_pred)
        precision = precision_score(y_val, y_pred)
        recall = recall_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred)
        roc_auc = roc_auc_score(y_val, y_pred_prob)

        metrics['accuracy'].append(accuracy)
        metrics['precision'].append(precision)
        metrics['recall'].append(recall)
        metrics['f1'].append(f1)
        metrics['roc_auc'].append(roc_auc)

        print(f"Fold Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}, ROC AUC: {roc_auc}")

    avg_metrics = {metric: np.mean(scores) for metric, scores in metrics.items()}
    print("\nAverage Metrics Across Folds:")
    for metric, avg_score in avg_metrics.items():
        print(f"{metric.capitalize()}: {avg_score:.4f}")
    
    return avg_metrics

# Example usage
# Load your data and tokenize it
texts = data["processed_full_content"]
num_samples = len(texts)
labels = data["label"].values  # Adjusted labels for each sample

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
X = tokenizer.texts_to_sequences(texts)
max_len = 100
X = pad_sequences(X, maxlen=max_len)

# Load GloVe embeddings
embedding_dim = 100
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = load_glove_embeddings("glove.6B.100d.txt", tokenizer.word_index, embedding_dim)

# Perform K-Fold cross-validation
avg_metrics = k_fold_cross_validation(X, labels, embedding_matrix, vocab_size, max_len)
print("Final Average Metrics:", avg_metrics)


Epoch 1/10
400/400 [==============================] - 12s 25ms/step - loss: 0.2395 - accuracy: 0.9117 - val_loss: 0.1950 - val_accuracy: 0.9338
Epoch 2/10
400/400 [==============================] - 10s 24ms/step - loss: 0.1219 - accuracy: 0.9623 - val_loss: 0.1260 - val_accuracy: 0.9599
Epoch 3/10
400/400 [==============================] - 10s 24ms/step - loss: 0.0825 - accuracy: 0.9771 - val_loss: 0.1347 - val_accuracy: 0.9592
Epoch 4/10
400/400 [==============================] - 10s 24ms/step - loss: 0.0633 - accuracy: 0.9845 - val_loss: 0.1148 - val_accuracy: 0.9650
Epoch 5/10
400/400 [==============================] - 9s 24ms/step - loss: 0.0517 - accuracy: 0.9884 - val_loss: 0.1281 - val_accuracy: 0.9630
Epoch 6/10
400/400 [==============================] - 10s 24ms/step - loss: 0.0391 - accuracy: 0.9927 - val_loss: 0.1309 - val_accuracy: 0.9633
Epoch 7/10
400/400 [==============================] - 2s 5ms/step
Fold Accuracy: 0.9650015659254619, Precision: 0.9632596204074525, Recal

# LSTM + GloVe 300D

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


# Load and process GloVe embeddings
def load_glove_embeddings(glove_file, word_index, embedding_dim=300):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefficients = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefficients
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix


# Define LSTM model with Dropout and L2 regularization
def create_lstm_model(vocab_size, embedding_matrix, input_length, learning_rate=0.001, l2_lambda=0.01):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_matrix.shape[1],
                        weights=[embedding_matrix],
                        input_length=input_length,
                        trainable=True))
    model.add(LSTM(units=64, return_sequences=False, dropout=0.2))
    model.add(Dropout(0.2))  # Added Dropout layer here
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(l2_lambda)))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate), metrics=['accuracy'])

    return model
early_stopping = EarlyStopping(
    monitor='val_loss',  # or 'val_accuracy' if you prefer to monitor accuracy
    patience=3,          # stop after 3 epochs with no improvement
    restore_best_weights=True  # revert to the best weights after stopping
)
# K-Fold Cross-Validation with additional metrics
def k_fold_cross_validation(X, y, embedding_matrix, vocab_size, max_len, n_splits=5, batch_size=128):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'roc_auc': []}

    # Define early stopping
    early_stopping = EarlyStopping(
        monitor='val_loss',  # monitor validation loss
        patience=3,          # number of epochs with no improvement
        restore_best_weights=True  # revert to the best model weights
    )

    for train_idx, val_idx in kfold.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        model = create_lstm_model(vocab_size, embedding_matrix, max_len, learning_rate=0.001, l2_lambda=0.01)
        
        # Fit model with early stopping
        model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=10,             # maximum number of epochs
            batch_size=128,
            verbose=1,
            callbacks=[early_stopping]  # add early stopping
        )
        
        y_pred = (model.predict(X_val) > 0.5).astype("int32")
        y_pred_prob = model.predict(X_val).ravel()

        accuracy = accuracy_score(y_val, y_pred)
        precision = precision_score(y_val, y_pred)
        recall = recall_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred)
        roc_auc = roc_auc_score(y_val, y_pred_prob)

        metrics['accuracy'].append(accuracy)
        metrics['precision'].append(precision)
        metrics['recall'].append(recall)
        metrics['f1'].append(f1)
        metrics['roc_auc'].append(roc_auc)

        print(f"Fold Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}, ROC AUC: {roc_auc}")

    avg_metrics = {metric: np.mean(scores) for metric, scores in metrics.items()}
    print("\nAverage Metrics Across Folds:")
    for metric, avg_score in avg_metrics.items():
        print(f"{metric.capitalize()}: {avg_score:.4f}")
    
    return avg_metrics

# Example usage
# Load your data and tokenize it
texts = data["processed_full_content"]
num_samples = len(texts)
labels = data["label"].values  # Adjusted labels for each sample

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
X = tokenizer.texts_to_sequences(texts)
max_len = 100
X = pad_sequences(X, maxlen=max_len)

# Load GloVe embeddings
embedding_dim = 300
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = load_glove_embeddings("glove.6B.300d.txt", tokenizer.word_index, embedding_dim)

# Perform K-Fold cross-validation
avg_metrics = k_fold_cross_validation(X, labels, embedding_matrix, vocab_size, max_len)
print("Final Average Metrics:", avg_metrics)


Epoch 1/10
400/400 [==============================] - 21s 48ms/step - loss: 0.2060 - accuracy: 0.9272 - val_loss: 0.1394 - val_accuracy: 0.9564
Epoch 2/10
400/400 [==============================] - 18s 46ms/step - loss: 0.0929 - accuracy: 0.9735 - val_loss: 0.1143 - val_accuracy: 0.9627
Epoch 3/10
400/400 [==============================] - 18s 46ms/step - loss: 0.0600 - accuracy: 0.9854 - val_loss: 0.1197 - val_accuracy: 0.9659
Epoch 4/10
400/400 [==============================] - 18s 46ms/step - loss: 0.0477 - accuracy: 0.9903 - val_loss: 0.1381 - val_accuracy: 0.9620
Epoch 5/10
400/400 [==============================] - 2s 4ms/step
Fold Accuracy: 0.9626526777325399, Precision: 0.9592084707516056, Recall: 0.9580443828016644, F1 Score: 0.9586260733801717, ROC AUC: 0.9936254723870884
Epoch 1/10
400/400 [==============================] - 20s 48ms/step - loss: 0.2095 - accuracy: 0.9246 - val_loss: 0.1202 - val_accuracy: 0.9634
Epoch 2/10
400/400 [==============================] - 19s 46ms

# Bidirectional LSTM + GloVe 300D

Instead of Standard LSTM from the previous code, which processes sequences in one direction, we implement Bidirectional LSTM which consists of 2 LSTM layers: one that processes sequences forward and one backward.

Bidirectional LSTM generally provides better performance on NLP tasks where context from both directions is essential. However Bidirectional LSTM requires double the memory and computational resources compared to a standard LSTM since it processes each sequence in two directions. Standard LSTM is more resource-efficient and faster.

In [5]:
import numpy as np
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


# Load and process GloVe embeddings
def load_glove_embeddings(glove_file, word_index, embedding_dim=300):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefficients = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefficients
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix


# Define LSTM model with Dropout and L2 regularization
def create_lstm_model(vocab_size, embedding_matrix, input_length, learning_rate=0.001, l2_lambda=0.01):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_matrix.shape[1],
                        weights=[embedding_matrix],
                        input_length=input_length,
                        trainable=True))
    model.add(Bidirectional(LSTM(units=64, return_sequences=False, dropout=0.2)))
    model.add(Dropout(0.2))  # Added Dropout layer here
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(l2_lambda)))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate), metrics=['accuracy'])

    return model
early_stopping = EarlyStopping(
    monitor='val_loss',  # or 'val_accuracy' if you prefer to monitor accuracy
    patience=3,          # stop after 3 epochs with no improvement
    restore_best_weights=True  # revert to the best weights after stopping
)
# K-Fold Cross-Validation with additional metrics
def k_fold_cross_validation(X, y, embedding_matrix, vocab_size, max_len, n_splits=5, batch_size=128):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'roc_auc': []}

    # Define early stopping
    early_stopping = EarlyStopping(
        monitor='val_loss',  # monitor validation loss
        patience=3,          # number of epochs with no improvement
        restore_best_weights=True  # revert to the best model weights
    )

    for train_idx, val_idx in kfold.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        model = create_lstm_model(vocab_size, embedding_matrix, max_len, learning_rate=0.001, l2_lambda=0.01)
        
        # Fit model with early stopping
        model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=10,             # maximum number of epochs
            batch_size=128,
            verbose=1,
            callbacks=[early_stopping]  # add early stopping
        )
        
        y_pred = (model.predict(X_val) > 0.5).astype("int32")
        y_pred_prob = model.predict(X_val).ravel()

        accuracy = accuracy_score(y_val, y_pred)
        precision = precision_score(y_val, y_pred)
        recall = recall_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred)
        roc_auc = roc_auc_score(y_val, y_pred_prob)

        metrics['accuracy'].append(accuracy)
        metrics['precision'].append(precision)
        metrics['recall'].append(recall)
        metrics['f1'].append(f1)
        metrics['roc_auc'].append(roc_auc)

        print(f"Fold Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}, ROC AUC: {roc_auc}")

    avg_metrics = {metric: np.mean(scores) for metric, scores in metrics.items()}
    print("\nAverage Metrics Across Folds:")
    for metric, avg_score in avg_metrics.items():
        print(f"{metric.capitalize()}: {avg_score:.4f}")
    
    return avg_metrics

# Example usage
# Load your data and tokenize it
texts = data["processed_full_content"]
num_samples = len(texts)
labels = data["label"].values  # Adjusted labels for each sample

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
X = tokenizer.texts_to_sequences(texts)
max_len = 100
X = pad_sequences(X, maxlen=max_len)

# Load GloVe embeddings
embedding_dim = 300
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = load_glove_embeddings("glove.6B.300d.txt", tokenizer.word_index, embedding_dim)

# Perform K-Fold cross-validation
avg_metrics = k_fold_cross_validation(X, labels, embedding_matrix, vocab_size, max_len)
print("Final Average Metrics:", avg_metrics)


Epoch 1/10
400/400 [==============================] - 27s 61ms/step - loss: 0.2000 - accuracy: 0.9268 - val_loss: 0.1316 - val_accuracy: 0.9569
Epoch 2/10
400/400 [==============================] - 23s 57ms/step - loss: 0.0917 - accuracy: 0.9730 - val_loss: 0.1197 - val_accuracy: 0.9630
Epoch 3/10
400/400 [==============================] - 23s 57ms/step - loss: 0.0567 - accuracy: 0.9855 - val_loss: 0.1224 - val_accuracy: 0.9627
Epoch 4/10
400/400 [==============================] - 23s 57ms/step - loss: 0.0377 - accuracy: 0.9921 - val_loss: 0.1399 - val_accuracy: 0.9626
Epoch 5/10
400/400 [==============================] - 3s 8ms/step
Fold Accuracy: 0.963044159098027, Precision: 0.9676792652772872, Recall: 0.9498959778085991, F1 Score: 0.9587051618547682, ROC AUC: 0.9925806464069273
Epoch 1/10
400/400 [==============================] - 26s 59ms/step - loss: 0.2046 - accuracy: 0.9244 - val_loss: 0.1310 - val_accuracy: 0.9579
Epoch 2/10
400/400 [==============================] - 23s 57ms/